In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
categories = {0: 'hate', 1: 'normal', 2: 'offensive'}

In [3]:
u = 'go fuck yourself asshole'
inputs = tokenizer(u, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
label = categories[np.argmax(outputs.logits.softmax(dim=-1).tolist())]

In [4]:
outputs.logits.softmax(dim=-1).tolist()

[[0.0731307715177536, 0.35753580927848816, 0.5693334341049194]]

In [5]:
label

'offensive'

In [6]:
def get_classifications_results(df):
    
    df = df.copy()
    labels_all = df.majority_answer.values
    predicted_all = df.label_hateexplain.values

    results_classification = classification_report(labels_all, predicted_all, output_dict=True)
        
    
    df_report = pd.DataFrame(results_classification).transpose()
    
    return df_report

In [7]:
def annotate_df(movie_df):
    result = []
    utterances = movie_df.text.values
    
    for i, u in enumerate(utterances):
        inputs = tokenizer(u, return_tensors="pt")
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(**inputs, labels=labels)
        label = categories[np.argmax(outputs.logits.softmax(dim=-1).tolist())]
        
        result.append({'index': i, 'label_hateexplain': label})
    
    result_df = pd.DataFrame(result)
    movie_df = movie_df.merge(result_df, right_index=True, left_index=True)
    
    return movie_df

In [8]:
path = r'E:\github\movie_hatespeech_detection\data\movies_for_training\all_movies.csv'
movie_data = pd.read_csv(path, index_col=0)

In [9]:
result_df = annotate_df(movie_data)

In [10]:
result_df['label_hateexplain'] = result_df['label_hateexplain'].replace({'normal': 0, 'offensive': 1, 'hate': 2})

In [11]:
result_df.label_hateexplain.value_counts(normalize=True)

0    0.943301
1    0.035647
2    0.021052
Name: label_hateexplain, dtype: float64

In [12]:
get_classifications_results(result_df).round(2)

,precision,recall,f1-score,support
0,0.88,0.98,0.93,9014.00
1,0.62,0.17,0.27,1380.00
2,0.89,0.68,0.77,294.00
accuracy,0.87,0.87,0.87,0.87
macro avg,0.80,0.61,0.66,10688.00
weighted avg,0.85,0.87,0.84,10688.00


In [13]:
result_df.label_hateexplain.value_counts(normalize=True)

0    0.943301
1    0.035647
2    0.021052
Name: label_hateexplain, dtype: float64

In [14]:
get_classifications_results(result_df).round(2)

,precision,recall,f1-score,support
0,0.88,0.98,0.93,9014.00
1,0.62,0.17,0.27,1380.00
2,0.89,0.68,0.77,294.00
accuracy,0.87,0.87,0.87,0.87
macro avg,0.80,0.61,0.66,10688.00
weighted avg,0.85,0.87,0.84,10688.00


In [15]:
0.93 + 0.27 + 0.77

1.9700000000000002

In [16]:
1.97 / 3

0.6566666666666666